In [1]:
%%capture
!pip install -Uqq fastai
!apt-get install tree

In [2]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from fastai.vision.all import *
from fastai import *
from torchvision import transforms
import pandas as pd

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
print('First 2 levels of the data directory... \n')
!tree -L 2 {str(path)}
# print('\nLooking at the first 10 files in the training/0 directory...\n')
!ls {str(path/'training/0')} | head -n 10

In [5]:
dls = ImageDataLoaders.from_folder(path, 'training', 'testing')

In [6]:
dls.show_batch()

In [7]:
x_b, y_b = dls.one_batch()

x_b.max(), x_b.min(), x_b.shape, y_b

In [8]:
def conv_block(in_channels, out_channels, kernel_size=3, padding=1, stride=1, dropout=0.2, *args, **kwargs):
    """
    Returns a sequential module of a conv block that we want to repeat.
    """
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, **kwargs),
        nn.PReLU(),
        nn.BatchNorm2d(out_channels),
        nn.Dropout2d(dropout)
    )

In [9]:
# Build our model
model = nn.Sequential(
    conv_block(3, 32), # 3x28x28 --> 32x28x28
    conv_block(32, 64), # 32x28x28 --> 64x28x28
    conv_block(64, 64, stride=2), # 64x28x28 --> 64x14x14
    conv_block(64, 128), # 64x14x14 --> 128x14x14
    conv_block(128, 128, stride=2), # 128x14x14 --> 128x7x7
    nn.AdaptiveMaxPool2d(1), # 128x7x7 --> 128
    nn.Flatten(), # 1x1x128 --> 128
    nn.Linear(128, 10) # 128 features --> 10 outputs
)

# Display our model
model

In [10]:
cbs = [
    EarlyStoppingCallback()
]

mets = [
    metrics.accuracy,
    metrics.error_rate
]

loss = CrossEntropyLossFlat()

In [11]:
learn = Learner(dls, model, loss, metrics=mets, cbs=cbs)

In [12]:
LR = 1e-3
learn.fit_one_cycle(2, LR)

In [13]:
learn.recorder.plot_sched()

In [14]:
def visualize_filters(layer, batch, batch_idx):
    with hook_output(model[layer][0]) as hook:
        # Pass one batch through the model, outputs will be stored in hook.stored
        with torch.no_grad():
            _ = learn.model(batch)
            # Store the outputs
            outputs = hook.stored
    
    # Pick an index in the batch
    n_filters = outputs.shape[1]
    output_size = outputs.shape[2]
    # Pick 9 random filters to visualize
    sampled_filters = np.random.choice(range(n_filters), 9)

    # Show the outputs for our 9 randomly sampled filters
    fig = plt.figure(figsize=(10, 10))
    for i, fidx in enumerate(sampled_filters):
        ax = fig.add_subplot(3, 3, 1+i)
        if i == 0: # Plot the original image in the upper left
            show_image(batch[batch_idx], title='Original', ax=ax)
            # plt.close()
        else: # Plot activation maps of the randomly sampled filters
            ax.matshow(
                outputs[batch_idx, fidx].squeeze().cpu().numpy(), 
                cmap='Greys_r'
            )
        # Get rid of ticks & labels
        ax.set_xticklabels([])
        ax.set_xticks([])
        ax.set_yticklabels([])
        ax.set_yticks([])
    fig.tight_layout()
    plt.close()
    return fig

In [15]:
visualize_filters(0, x_b, 2)

In [16]:
visualize_filters(2, x_b, 2)

In [17]:
visualize_filters(4, x_b, 2)